In [1]:
using TSSM
include("time_stepper.jl");

In [2]:
function get_coeffs_composition(g::Vector{Float64})
    n = length(g)
    z = zeros(n)
    a = zeros(n+1)
    b = vcat(g, 0)
    h = 0.0
    z[div(n,2)+1] = 0.5
    for j=1:div(n,2)
        z[j] = h+g[j]/2
        z[n-j+1] = 1-z[j]
        h += g[j]
    end
    a[1] = g[1]/2
    a[n+1] = a[1]
    for j=2:div(n,2)+1
        a[j] = (g[j-1]+g[j])/2
        a[n-j+2] = a[j]
    end
    a,b,z
end

get_coeffs_composition (generic function with 1 method)

In [3]:
function gen_interpolation_matrix(x::Vector{Float64}, z::Vector{Float64})
    n = length(x)
    m = length(z)    
    L = zeros(n, m)
    for j=1:m
        for i=1:n
            L[i,j] = prod([(z[j]-x[k])/(x[i]-x[k]) for k=1:i-1])*prod([(z[j]-x[k])/(x[i]-x[k]) for k=i+1:n])
        end
    end
    L
end

gen_interpolation_matrix (generic function with 1 method)

In [84]:
function gen_midpoint!(midpoint::WfSchroedinger1D, L::Vector{Float64}, psi_back::Vector{WfSchroedinger1D}, first::Int)
    n = length(psi_back)
    set!(midpoint, 0)
    for j=1:n
        k = mod(j+first-2, n)+1
        axpy!(midpoint, psi_back[k], L[j])
    end
end

gen_midpoint! (generic function with 1 method)

In [114]:
function TSSM.propagate_B!(psi::WfSchroedinger1D, dt::Number, F::Function, t::Number, midpoint::WfSchroedinger1D)
    to_real_space!(psi)
    to_real_space!(midpoint)
    u = get_data(psi, true)
    u1 = get_data(midpoint, true)
    x = get_nodes(psi.m)
    for j=1:length(u)
        u[j] += dt*F(t, x[j], u1[j])
    end
end

propagate_B! (generic function with 23 methods)

In [164]:
function run(F::Function, dt::Number, N::Int, psi_back::Vector{WfSchroedinger1D}, 
    a::Vector{Float64}, b::Vector{Float64}, z::Vector{Float64})
    n = length(psi_back)
    m = psi_back[1].m
    psi = wave_function(m)
    midpoint = wave_function(m)
    copy!(psi, psi_back[n])
    s = length(a)
    L = gen_interpolation_matrix(collect(-(n-1.0):0.0), z)
    first = 1
    for k = n:N
        for j = 1:s
            propagate_A!(psi, a[j]*dt)            
            if b[j]!=0.0
                gen_midpoint!(midpoint, L[:,j], psi_back, first)
                propagate_B!(psi, b[j]*dt, F, get_time(psi), midpoint)
            end
        end    
        copy!(psi_back[first], psi)
        to_real_space!(psi_back[first])
        first = mod(first, n) + 1
    end
    psi
end

run (generic function with 2 methods)

In [161]:
run(F::Function, dt::Number, N::Int, psi_back::Vector{WfSchroedinger1D}, g::Vector{Float64}) = run(
F, dt, N, psi_back, get_coeffs_composition(g)...)

run (generic function with 2 methods)

In [162]:
function gen_starting_values(psi::WfSchroedinger1D, dt::Number, N::Int, 
    a::Vector{Float64}, b::Vector{Float64}, z::Vector{Float64})
    s = length(a)
    psi_back = WfSchroedinger1D[wave_function(m) for j=1:N]
    copy!(psi_back[1], psi)
    for k=2:N
        for j = 1:s
            propagate_A!(psi, a[j]*dt)
            if b[j]!=0.0
                propagate_B!(psi, b[j]*dt)
            end
        end
        copy!(psi_back[k], psi)
    end
    psi_back
end

gen_starting_values (generic function with 2 methods)

In [170]:
gen_starting_values(psi::WfSchroedinger1D, dt::Number, N::Int, g::Vector{Float64}) = gen_starting_values(
psi,dt, N, get_coeffs_composition(g)...)

gen_starting_values (generic function with 2 methods)

In [117]:
nx = 8192
xmin = -16
xmax = +16
m = Schroedinger1D(nx, xmin, xmax, cubic_coupling=-1)

TSSM.Schroedinger1D{Float64}(Ptr{Void} @0x0000000006edbaa0)

In [118]:
# cubic_coupling=-1 has to be multiplied by -1im 
# because of the factor 1im at the lefthand side of the Schrödinger equation
F(t, x, u) = 1im*conj(u)*u^2

F (generic function with 3 methods)

In [119]:
# exact solution
const a = 2
const b = 1
const c = 0
function soliton(x, t)
    h = (a^2 - b^2)/2*t - b*x
    (a./cosh(a*(b*t+x-c))).*exp(1im*h) 
end    

soliton (generic function with 1 method)

In [120]:
t0 = 0
tend = 1
psi = wave_function(m)
psi_ref = wave_function(m)
set!(psi, soliton, t0)        # initial data at t=t0
set!(psi_ref, soliton, tend)  # reference solution at t=tend

Composition coefficients from
http://www.netlib.org/ode/composition.txt
see also
http://www.ams.org/journals/mcom/1997-66-219/S0025-5718-97-00873-9/S0025-5718-97-00873-9.pdf

In [121]:
s9odr6a=[ 
.39216144400731413928,
.33259913678935943860,
-.70624617255763935981,
.82213596293550800230E-1,
.79854399093482996340,
.82213596293550800230E-1,
-.70624617255763935981,
.33259913678935943860,
.39216144400731413928,

]

9-element Array{Float64,1}:
  0.392161 
  0.332599 
 -0.706246 
  0.0822136
  0.798544 
  0.0822136
 -0.706246 
  0.332599 
  0.392161 

In [122]:
s9odr6b=[
.39103020330868478817,
.33403728961113601749,
-.70622728118756134345,
.81877549648059445772E-1,
.79856447723936218405,
.81877549648059445772E-1,
-.70622728118756134345,
.33403728961113601749,
.39103020330868478817]

9-element Array{Float64,1}:
  0.39103  
  0.334037 
 -0.706227 
  0.0818775
  0.798564 
  0.0818775
 -0.706227 
  0.334037 
  0.39103  

For the following see http://www.tandfonline.com/doi/abs/10.1080/10556780500140664

In [123]:
s11odr6=[
0.21375583945878254555518066964857,
0.18329381407425713911385974425217,
0.17692819473098943794898811709929,
-0.44329082681170215849622829626258,
0.11728560432865935385403585669136,
0.50405474843802736404832781714239,
0.11728560432865935385403585669136,
-0.44329082681170215849622829626258,
0.17692819473098943794898811709929,
0.18329381407425713911385974425217,
0.21375583945878254555518066964857]    

11-element Array{Float64,1}:
  0.213756
  0.183294
  0.176928
 -0.443291
  0.117286
  0.504055
  0.117286
 -0.443291
  0.176928
  0.183294
  0.213756

In [124]:
set!(psi, soliton, t0)
#g = [1/(2-2^(1/3)),-2^(1/3)/(2-2^(1/3)), 1/(2-2^(1/3))] # Yoshida
g = [1/(4-4^(1/3)),1/(4-4^(1/3)),-4^(1/3)/(4-4^(1/3)), 1/(4-4^(1/3)), 1/(4-4^(1/3))] # Suzuki
#g = s11odr6
N = 256
dt = (tend-t0)/N
psi_back = gen_starting_values(psi, dt, 4, g);
psi = run(F, dt, N, psi_back, g)
distance(psi, psi_ref)

1.7002608394763918e-7

In [104]:
1.700260839476392e-7/7.416825153607977e-10

229.24375379798266

In [69]:
2.9801973871570312e-5/5.873375783588756e-7

50.7407919562073

In [14]:
5.56784149074978e-9/3.507350738832297e-10

15.874778159778462

In [125]:
const α=0.5
#V2(x,t) = (α-1)*abs(soliton(x,t)).^2
V2(x,t) = (α-1)*a^2./cosh(a*(b*t+x-c)).^2
m = Schroedinger1D(nx, xmin, xmax, potential_t=V2, cubic_coupling=-α)
psi = wave_function(m)
psi_ref = wave_function(m)
set!(psi, soliton, t0)        # initial data at t=t0
set!(psi_ref, soliton, tend)  # reference solution at t=tend

In [136]:
F(t, x, u) = 1im*(α*conj(u)*u + (1-α)*a^2./cosh(a*(b*t+x-c)).^2)*u

F (generic function with 3 methods)

In [171]:
set!(psi, soliton, t0)
#g = [1/(2-2^(1/3)),-2^(1/3)/(2-2^(1/3)), 1/(2-2^(1/3))] # Yoshida
#g = [1/(4-4^(1/3)),1/(4-4^(1/3)),-4^(1/3)/(4-4^(1/3)), 1/(4-4^(1/3)), 1/(4-4^(1/3))] # Suzuki
g = s11odr6
N = 512
dt = (tend-t0)/N
psi_back = gen_starting_values(psi, dt, 6, g);
set_time!(psi, t0)
psi = run(F, dt, N, psi_back, g)
distance(psi, psi_ref)

3.183650087680294e-12

In [177]:
a_i =[
 1.000000000000000000,
-0.666666666666666667,
    0.666666666666666667]
b_i=[
-0.0416666666666666667,
 0.750000000000000000,
    0.291666666666666667]

3-element Array{Float64,1}:
 -0.0416667
  0.75     
  0.291667 

In [ ]:
a_i =[
    0.268330095781759925,
    -0.187991618799159782,
    0.919661523017399857]
b_i =[
 0.919661523017399857,
-0.187991618799159782,
    0.268330095781759925]

In [187]:
a_i=[
 0.201651044312324230,
 0.562615975356569200,
 0.253874038247554845,
-0.835351693190370636,
 0.068014946093165092,
-0.102733803148432142,
 0.273128836056524479,
0.578800656272664932]
b_i=[
 0.578800656272664932,
 0.273128836056524479,
-0.102733803148432142,
 0.068014946093165092,
-0.835351693190370636,
 0.253874038247554845,
 0.562615975356569200,
0.201651044312324230]

8-element Array{Float64,1}:
  0.578801 
  0.273129 
 -0.102734 
  0.0680149
 -0.835352 
  0.253874 
  0.562616 
  0.201651 

In [189]:
z = cumsum(a_i)
set!(psi, soliton, t0)
N = 256
dt = (tend-t0)/N
psi_back = gen_starting_values(psi, dt, 5, a_i, b_i, z);
set_time!(psi, t0)
psi = run(F, dt, N, psi_back, a_i, b_i, z)
distance(psi, psi_ref)

1.1147852675915409e-9

In [190]:
3.403355261266454e-8/1.1147852675915409e-9

30.529245050208615

In [176]:
3.636730689666147e-6/4.6036650372580443e-7

7.899642263791185

In [181]:
3.692930808435428e-6/4.6753349232228637e-7

7.898751360233607